In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path of folder
folder_path = '/content/drive/MyDrive/Colab Notebooks/2024_08_01 for all sites kmeans site id wise last 3 months /'

# Get list of all CSV files in the folder
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty list to hold the dataframes
dfs = []

# Loop through the CSV files and read each one into a dataframe
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)



Mounted at /content/drive


<ipython-input-1-02eea4cc2222>:28: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [ ]:
# Concatenate all dataframes into a single dataframe
data = pd.concat(dfs, ignore_index=True)

# Sort by 'First Occurred On' in descending order (latest alarms first)
df = data.sort_values(by='First Occurred On', ascending=False)

# Convert 'First Occurred On' to datetime, handle errors and fill NaT with a default date
df['First Occurred On'] = pd.to_datetime(df['First Occurred On'], errors='coerce')

In [ ]:
# Reference time
reference_time = pd.to_datetime('2024-08-01 22:00:00')

# Calculate the relative day index, handle NaT by filling with a large negative number
df['Relative Day Index'] = ((df['First Occurred On'] - reference_time).dt.total_seconds() // 86400).fillna(-9999).astype(int)

# Adjust the index to set the 08:00 to 08:00 of the next day as 0, and previous days as -1, -2, -3, etc.
df['Relative Day Index'] = df['Relative Day Index'].apply(lambda x: x if x < 0 else x)

# Print the updated dataframe
df.head(50000)

,Last Occurred On,Alarm Source,Alarm Severity,Alarm Name,Alarm ID,Alarm Location Info,Site Name,Cleared On,First Occurred On,Clearance Status,Ticket ID,Site ID,Vendor,Domain,Device Type,Relative Day Index,Cluster
3864384,2024-08-01 23:03:12,Badulla-BD0001-L,Major,User Plane Fault,25954,Service Type=X2,Badulla,NaN,2024-08-01 23:03:12,Uncleared,NaN,BD0001,HUAWEI,Access,eNodeB,0,NaN
3864385,2024-08-01 23:02:44,MC1_Mundal_South-PU0271-BDGHP,Critical,Battery Deep Discharge Alarm,65089,"Cabinet No.=0, Subrack No.=0, Slot No.=19, Por...",Mundal_South,NaN,2024-08-01 23:02:44,Uncleared,NaN,PU0271,HUAWEI,Power,GBTS,0,NaN
3864386,2024-08-01 23:02:36,vUSN02,Major,S1ap Link Down,80589,"Mobile Country Code=413, Mobile Network Code=0...",NaN,2024-08-01 23:08:50,2024-08-01 23:02:36,Cleared,NaN,NaN,HUAWEI,Others,NaN,0,NaN
3864387,2024-08-01 23:02:30,PILI-AMF-01,Major,S1ap Link Down,80589,"Service Instance=LINK_VNFC_999, Mobile Country...",Piliyandala,2024-08-01 23:08:54,2024-08-01 23:02:30,Cleared,NaN,CM0091,HUAWEI,Others,UNC,0,NaN
3864388,2024-08-01 23:02:27,PILI_VUSN01,Major,S1ap Link Down,80589,"Mobile Country Code=413, Mobile Network Code=0...",NaN,2024-08-01 23:08:51,2024-08-01 23:02:27,Cleared,NaN,NaN,HUAWEI,Others,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47722,2024-07-31 12:06:24,U2020ACC,Major,Parallel Alarm Exceeds the Limit,832,"RuleID=1984, Source Alarm Information=""Alarm N...",NaN,2024-07-31 12:50:25,2024-07-31 12:06:24,Cleared,NaN,NaN,HUAWEI,Others,NaN,-2,NaN
47723,2024-07-31 12:06:23,U2020ACC,Major,Parallel Alarm Exceeds the Limit,832,"RuleID=1980, Source Alarm Information=""Alarm N...",NaN,2024-07-31 12:50:25,2024-07-31 12:06:23,Cleared,NaN,NaN,HUAWEI,Others,NaN,-2,NaN
47724,2024-07-31 12:06:21,PILI-AMF-01,Major,S1ap Link Down,80589,"Service Instance=LINK_VNFC_999, Mobile Country...",Piliyandala,2024-07-31 17:10:15,2024-07-31 12:06:21,Cleared,NaN,CM0091,HUAWEI,Others,UNC,-2,NaN
47725,2024-07-31 12:06:17,PILI_VUSN01,Major,S1ap Link Down,80589,"Mobile Country Code=413, Mobile Network Code=0...",NaN,2024-07-31 17:10:15,2024-07-31 12:06:17,Cleared,NaN,NaN,HUAWEI,Others,NaN,-2,NaN


In [ ]:
# Filter the data for HUAWEI and Access domain
filtered_data = df[(df['Vendor'] == 'HUAWEI')]

# Retain only the specified columns
columns_to_keep = ['Site ID', 'Alarm Name', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index']
df_filtered = filtered_data[columns_to_keep]

# Identify the five most recent Site IDs
recent_site_ids = df_filtered['Site ID'].value_counts().index[:5]

# Filter the dataframe to include only the most recent Site IDs
df_filtered = df_filtered[df_filtered['Site ID'].isin(recent_site_ids)]

# Encode categorical columns except 'Relative Day Index'
categorical_columns = df_filtered.select_dtypes(include=['object']).columns
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    df_filtered[column] = le.fit_transform(df_filtered[column].astype(str))
    label_encoders[column] = le

# Print the updated dataframe
df_filtered.head(100)

,Site ID,Alarm Name,Vendor,Domain,Device Type,Relative Day Index
3864387,0,459,0,4,18,0
3864391,4,229,0,2,10,0
3864397,4,230,0,2,22,0
3864406,0,459,0,4,18,0
3864414,1,248,0,4,6,0
...,...,...,...,...,...,...
276,0,459,0,4,18,0
282,4,459,0,2,13,0
280,4,459,0,2,13,0
279,4,459,0,2,13,0


In [ ]:
# Clustering to find common patterns
# Determine the optimal number of clusters using the elbow method
# wcss = []
# for i in range(1, 11):
#     kmeans = KMeans(n_clusters=i, random_state=42)
#     kmeans.fit(df_filtered)
#     wcss.append(kmeans.inertia_)

# plt.plot(range(1, 11), wcss)
# plt.title('Elbow Method for Optimal k')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

# Fit the KMeans model with the optimal number of clusters
optimal_clusters =20   # Increased number of clusters
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
df_filtered['Cluster'] = kmeans.fit_predict(df_filtered[['Alarm Name', 'Site ID']])

# Analyze the clusters
print(df_filtered.groupby('Cluster').mean())
print(df_filtered.groupby('Cluster').size())

# Print cluster details and alarm patterns
cluster_patterns = {}
for cluster in range(optimal_clusters):
    print(f"Cluster {cluster} details:")
    cluster_data = df_filtered[df_filtered['Cluster'] == cluster]
    alarm_patterns = cluster_data.groupby(['Alarm Name', 'Site ID']).size()
    cluster_patterns[cluster] = alarm_patterns
    print("Alarm Patterns:")
    print(alarm_patterns)
    print("\n")

# Plot the clusters
# sns.pairplot(df_filtered, hue='Cluster')
# plt.show()

# Inverse transform the scaled columns for interpretation
# scaler = MinMaxScaler()
# df_filtered[['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster']] = scaler.inverse_transform(df_filtered[['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster']])

# Convert back to original categorical values using label_encoders
# for column in label_encoders.keys():
#     df_filtered[column] = label_encoders[column].inverse_transform(df_filtered[column].round().astype(int))

# Display sample output of clustered data
df_filtered.head(100)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


          Site ID  Alarm Name  Vendor    Domain  Device Type  \
Cluster                                                        
0        0.632023  302.737261     0.0  2.216414    13.934457   
1        3.999979  459.036460     0.0  1.999773    17.134283   
2        2.072113   65.470249     0.0  2.843426    20.621075   
3        1.414391  144.324210     0.0  2.131540    13.791392   
4        3.864954  229.053326     0.0  2.044976    21.667415   
5        0.718422  328.108389     0.0  2.231597    13.934354   
6        2.764519  401.357622     0.0  1.727477     9.228366   
7        1.886066  555.379271     0.0  2.448622    18.428934   
8        1.440680   10.896672     0.0  2.097048    25.311085   
9        1.041934   95.783265     0.0  2.766738    24.953011   
10       0.807489   33.538015     0.0  2.811897    24.582460   
11       3.919869  496.867354     0.0  1.886593    10.522714   
12       2.941496  249.013424     0.0  2.111629    10.673680   
13       1.580387  365.819244     0.0  2

,Site ID,Alarm Name,Vendor,Domain,Device Type,Relative Day Index,Cluster
3864387,0,459,0,4,18,0,19
3864391,4,229,0,2,10,0,4
3864397,4,230,0,2,22,0,4
3864406,0,459,0,4,18,0,19
3864414,1,248,0,4,6,0,12
...,...,...,...,...,...,...,...
276,0,459,0,4,18,0,19
282,4,459,0,2,13,0,1
280,4,459,0,2,13,0,1
279,4,459,0,2,13,0,1


In [ ]:
# Re-scale the dataframe for LSTM
scaler = MinMaxScaler()
df_filtered[['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster']] = scaler.fit_transform(df_filtered[['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster']])

# Prepare the dataset for LSTM
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 10
features = df_filtered[['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster']].values
X, y = create_dataset(features, time_steps)


In [ ]:

# Reshape input to be [samples, time steps, features]
X = X.reshape(X.shape[0], time_steps, X.shape[2])

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, X.shape[2])))
model.add(LSTM(50))
model.add(Dense(X.shape[2]))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=1, batch_size=2048, verbose=1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


779/779 ━━━━━━━━━━━━━━━━━━━━ 159s 200ms/step - loss: 0.0627


In [ ]:
# Predict for the next 7 days (each day has 24 hours)
predictions = []
current_batch = features[-time_steps:].reshape((1, time_steps, X.shape[2]))

for i in range(24 * 7):  # 24 hours * 7 days
    pred = model.predict(current_batch, verbose=0)[0]
    predictions.append(pred)
    current_batch = np.append(current_batch[:, 1:, :], [[pred]], axis=1)

# Inverse transform the predictions
predictions = scaler.inverse_transform(predictions)


In [ ]:
# Create a DataFrame for predictions
relative_day_indices = np.arange(df_filtered['Relative Day Index'].max() + 1, df_filtered['Relative Day Index'].max() + 1 + 24 * 7)
pred_df = pd.DataFrame(predictions, columns=['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster'])
pred_df['Relative Day Index'] = relative_day_indices

# Convert back to original categorical values using label_encoders
for column in label_encoders.keys():
    pred_df[column] = pred_df[column].round().astype(int)
    pred_df[column] = label_encoders[column].inverse_transform(pred_df[column])

# Filter predictions for the next 7 days only
pred_df = pred_df[pred_df['Relative Day Index'] <= 7]

# Display sample output of predictions with cluster details
pred_df.head(7)

,Alarm Name,Site ID,Vendor,Domain,Device Type,Relative Day Index,Cluster
0,Power Module Stopped Working,GM0257,HUAWEI,Core,UDG,2.0,4.726676
1,Performance Threshold CS CORE,GM0257,HUAWEI,Core,UDG,3.0,5.412091
2,Performance Threshold Default Alarm,CM2074,HUAWEI,Core,UDG,4.0,5.762794
3,Performance Measurement Entity Value Exceeds L...,CM2074,HUAWEI,Core,UDG,5.0,5.802648
4,Peer Entity Capability Is Not Configured,CM2074,HUAWEI,Core,UDG,6.0,5.722408
5,Peer Entity Capability Is Not Configured,CM2074,HUAWEI,Core,UDG,7.0,5.663376


In [ ]:
# Filter predictions to include only Access domain alarms
pred_df_access = pred_df[pred_df['Domain'] == label_encoders['Domain'].transform(['Access'])[0]]

# Display sample output of Access domain predictions with cluster details
print(pred_df_access.head(7))

# Print the cluster patterns for the predicted alarms
for index, row in pred_df_access.iterrows():
    cluster = int(row['Cluster'])
    print(f"Predicted Alarm for Relative Day Index {row['Relative Day Index']}:")
    print(f"Cluster {cluster} Alarm Patterns:")
    alarm_names = cluster_patterns[cluster].index.get_level_values(0).map(lambda x: label_encoders['Alarm Name'].inverse_transform([x])[0])
    print(pd.Series(cluster_patterns[cluster].values, index=alarm_names))
    print("\n")

# Print cluster-wise alarm patterns
for cluster in range(optimal_clusters):
    print(f"Cluster {cluster} Alarm Patterns:")
    alarm_names = cluster_patterns[cluster].index.get_level_values(0).map(lambda x: label_encoders['Alarm Name'].inverse_transform([x])[0])
    print(pd.Series(cluster_patterns[cluster].values, index=alarm_names))
    print("\n")


Empty DataFrame
Columns: [Alarm Name, Site ID, Vendor, Domain, Device Type, Relative Day Index, Cluster]
Index: []
Cluster 0 Alarm Patterns:
Alarm Name
Login Attempts Reach the Maximum Value               1
Login Attempts Reach the Maximum Value               1
Loop occurs due to TTL timeout                       2
Loss of the reference source                        24
Lost synchronization of a TDM high-speed link        6
Low Fuel Alarm                                       3
M2PA Link Fault                                     24
M2PA Link Fault                                     24
M2PA SCTP Path Fault                                17
M2PA SCTP Path Fault                                16
M2UA Link out of service                             5
M2UA LinkSet out of service                          3
M2UA SCTP Path Alarm                                 5
M3UA Application Server Inaccessible             43856
M3UA Application Server Inaccessible             43866
M3UA Destination Entity

In [ ]:
# Display sample output of predictions with cluster details
print(pred_df_access.head(7))

Empty DataFrame
Columns: [Alarm Name, Site ID, Vendor, Domain, Device Type, Relative Day Index, Cluster]
Index: []
